In [23]:
import importlib.util
import sys
spec = importlib.util.spec_from_file_location("jupyter_drawing_pad", "../../../jupyter-drawing-pad/jupyter-drawing-pad/jupyter_drawing_pad/example.py")


In [32]:
from ipywidgets import HBox, VBox, Button, Text
from IPython import display
import numpy as np

In [33]:
foo = importlib.util.module_from_spec(spec)
sys.modules["jupyter_drawing_pad"] = foo
spec.loader.exec_module(foo)

In [35]:
class CustomBox(HBox):
    def __init__(self):
        drawing_pad = foo.DrawingPad()
        button = Button(description="Clear", tooltip="Click me")
        save_button = Button(description="Save", tooltip="Click me")
        login_button = Button(description="Login", tooltip="Click me")
        text_area = Text(value='', placeholder='Type your name', description='Name:', disabled=False)
        button.on_click(lambda b : self.clean())
        save_button.on_click(lambda b : self.set_saved())
        login_button.on_click(lambda b : self.check())
        buttons = VBox([text_area, button, save_button, login_button])
        self.drawing_pad = drawing_pad
        self.text_area = text_area
        self.__saved = {}
        super().__init__([drawing_pad, buttons])

    def clean(self):
        self.drawing_pad.clear()
        self.text_area.value=""

    def is_empty(self):
        return len(self.drawing_pad.data[0])==0

    def set_saved(self):
        name = self.text_area.value
        self.__saved[name] = self.drawing_pad.data
        self.clean()
        print('Your signature was saved, ' + str(name))
    
    def get_saved(self):
        return self.__saved
    
    def renormalize(self,t):
        t = (t-t[0])/(t[-1]-t[0])
        return t

    def value(self,t, list_x, list_y, list_t):
        idx = (np.abs(list_t-t)).argmin()
        return np.array([list_x[idx], list_y[idx]])

    def value_der(self,t, list_x, list_y, list_t):
        idx = (np.abs(list_t-t)).argmin()
        return np.array([(list_x[idx]-list_x[idx-1])/(list_t[idx]-list_t[idx-1]), 
                        (list_y[idx]-list_y[idx-1])/(list_t[idx]-list_t[idx-1])])

    def distance(self,x, y):
        return np.linalg.norm(x-y)

    def distance_total(self,list_x, list_y, list_t, list_x_prime, list_y_prime, list_t_prime):
        grid_time = np.linspace(0, 1, 100)
        value_tot = sum([self.distance(self.value(t, list_x, list_y, list_t), self.value(t, list_x_prime, list_y_prime, list_t_prime)) for t in grid_time])
        value_der = sum([self.distance(self.value_der(t, list_x, list_y, list_t), self.value_der(t, list_x_prime, list_y_prime, list_t_prime)) for t in grid_time])
        # print(value_der)
        return value_tot + value_der / 15

    def normalize_all(self,x,y,t):
        x = np.array(x)
        y = np.array(y)
        t = np.array(t)
        list_t_norm = self.renormalize(t)
        list_x_norm = (x - np.mean(x))/(np.std(x))
        list_y_norm = (y - np.mean(y))/(np.std(y))
        return (list_x_norm, list_y_norm, list_t_norm)


In [ ]:
widget = CustomBox()
widget.drawing_pad

In [ ]:
display(widget)

In [37]:
import ipywidgets as widgets

In [38]:
int_range = widgets.IntSlider(
    value=7,
    min=0,
    max=0x3f,
    step=1,
    description='Test:',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

In [ ]:
int_range

In [25]:
%gui tk
import tkinter as tk
from tkinter import ttk
import numpy as np
from PIL import Image, ImageDraw

In [42]:
import socket
send_address = ('192.168.0.180', 12345)
sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)

In [ ]:
# Initialize main window
root = tk.Tk()
root.title("Tkinter Drawing Pad")

# Global variables
brush_size = 30  # Default brush size

def paint(event):
    """Draw on the canvas with the current brush size."""
    x1, y1 = (event.x - brush_size), (event.y - brush_size)
    x2, y2 = (event.x + brush_size), (event.y + brush_size)
    canvas.create_oval(x1, y1, x2, y2, fill='white', outline='white')

def canvas_to_array(canvas):
    """Convert the canvas content to a NumPy array without Ghostscript."""
    # Create a blank white image the same size as the canvas
    canvas_width = canvas.winfo_width()
    canvas_height = canvas.winfo_height()
    image = Image.new("RGB", (canvas_width, canvas_height), "black")
    draw = ImageDraw.Draw(image)

    # Iterate through canvas items and draw them on the image
    for item in canvas.find_all():
        coords = canvas.coords(item)
        color = canvas.itemcget(item, "fill")
        draw.ellipse(coords, fill=color, outline=color)

    # Resize the image to (29, 29)
    image = image.resize((28, 28))

    # Convert the image to a NumPy array and reorder to (3, 29, 29)
    np_array = np.array(image).astype(np.uint8)
    #np_array = np_array.transpose(2, 0, 1)  # Convert to (3, 29, 29)

    
    return np_array

def save_as_numpy():
    """Save the canvas content as a NumPy array and print its shape."""
    np_array = canvas_to_array(canvas)  # Convert to NumPy array
    print("Numpy Array Shape:", np_array.shape)
    print(np_array[0][0][0])  # Print for debugging

    # Ensure the array is C-contiguous (to avoid errors)
    np_array = np.ascontiguousarray(np_array)

    # Convert the array to bytes
    data = np_array.tobytes()
    sock.sendto(data, send_address)

def clear_canvas():
    """Clear all drawings on the canvas."""
    canvas.delete("all")  # Remove all items from the canvas

def quit_app():
    """Exit the application."""
    root.destroy()  # Stop the event loop and close the window
    root.quit()

# Create a canvas to draw on
canvas = tk.Canvas(root, bg='black', width=500, height=500)
canvas.pack()

# Bind the paint function to left mouse button clicks and movement
canvas.bind("<B1-Motion>", paint)

# Save button to convert the drawing to a NumPy array
save_button = tk.Button(root, text="Save as NumPy Array", command=save_as_numpy)
save_button.pack()

# Clear button to reset the canvas
clear_button = tk.Button(root, text="Clear Canvas", command=clear_canvas)
clear_button.pack()

# Quit button to exit the application
quit_button = tk.Button(root, text="Quit", command=quit_app)
quit_button.pack()

# Run the Tkinter event loop
root.mainloop()

Numpy Array Shape: (3, 28, 28)
0
Numpy Array Shape: (3, 28, 28)
0
Numpy Array Shape: (3, 28, 28)
255
Numpy Array Shape: (3, 28, 28)
0
Numpy Array Shape: (3, 28, 28)
0
Numpy Array Shape: (3, 28, 28)
0
Numpy Array Shape: (3, 28, 28)
0
Numpy Array Shape: (3, 28, 28)
0
Numpy Array Shape: (3, 28, 28)
0
